In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [ ]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, RMSprop

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-04-10 00:26:59.013688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-10 00:26:59.013734: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [21]:

nActions = 4
tableSize = 4

# hiddenFeatures = 16
hiddenFeatures = 32
hiddenLayers = 1
# hiddenLayers = 10

activation="relu"
# activation="leaky_relu"

# Higher temporal window confuses the model
# temporalWindow = 5
temporalWindow = 1


env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

for i in range(hiddenLayers):
    model.add(Dense(hiddenFeatures))
    model.add(Activation(activation=activation))

# Result look alike
# model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha

# Worst result
# model.add(Activation(activation="elu")) # Tune alpha

# Output layer
model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()

# Less invalid moves with more limit
replayMemory = SequentialMemory(limit=2000, window_length=temporalWindow)

dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    # target_model_update=0.01,
    target_model_update=500,
    policy=policy,
)
# dqn.compile(optimizer=Adam(learning_rate=0.001), metrics=["mae"])
# dqn.compile(optimizer=Adam(learning_rate=0.001), metrics=["mse"])
# dqn.compile(optimizer=RMSprop(learning_rate=0.001, epsilon=0.0000001), metrics=["mae"])
# dqn.compile(optimizer=RMSprop(learning_rate=0.001, epsilon=0.0000001), metrics=["mse"])


In [22]:
dqn.compile(optimizer=RMSprop(learning_rate=0.01, epsilon=0.0000001), metrics=["mse"])
dqn.fit(env=env, nb_steps=2000, visualize=False, verbose=2)

dqn.compile(optimizer=RMSprop(learning_rate=0.001, epsilon=0.0000001), metrics=["mse"])
dqn.fit(env=env, nb_steps=10000, visualize=False, verbose=2)

dqn.compile(optimizer=RMSprop(learning_rate=0.0001, epsilon=0.0000001), metrics=["mse"])
dqn.fit(env=env, nb_steps=10000, visualize=False, verbose=2)

# dqn.compile(optimizer=RMSprop(learning_rate=0.00001, epsilon=0.0000001), metrics=["mse"])
# dqn.fit(env=env, nb_steps=10000, visualize=False, verbose=2)

Training for 2000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  349/2000: episode: 1, duration: 0.443s, episode steps: 349, steps per second: 789, episode reward: -12220.000, mean reward: -35.014 [-50.000, 68.000], mean action: 1.628 [0.000, 3.000],  loss: --, mse: --, mean_q: --
  652/2000: episode: 2, duration: 0.196s, episode steps: 303, steps per second: 1549, episode reward: -8698.000, mean reward: -28.706 [-50.000, 164.000], mean action: 1.904 [0.000, 3.000],  loss: --, mse: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1126/2000: episode: 3, duration: 1.420s, episode steps: 474, steps per second: 334, episode reward: -16024.000, mean reward: -33.806 [-50.000, 140.000], mean action: 1.080 [0.000, 3.000],  loss: 376.771841, mse: 603.668831, mean_q: -11.507554
 1262/2000: episode: 4, duration: 0.616s, episode steps: 136, steps per second: 221, episode reward: -458.000, mean reward: -3.368 [-50.000, 128.000], mean action: 1.941 [0.000, 3.000],  loss: 291.103729, mse: 533.470337, mean_q: 3.498658
 1414/2000: episode: 5, duration: 0.674s, episode steps: 152, steps per second: 225, episode reward: -2702.000, mean reward: -17.776 [-50.000, 84.000], mean action: 1.836 [0.000, 3.000],  loss: 254.180740, mse: 549.038879, mean_q: 12.747724
 1775/2000: episode: 6, duration: 1.575s, episode steps: 361, steps per second: 229, episode reward: -6256.000, mean reward: -17.330 [-50.000, 272.000], mean action: 1.443 [0.000, 3.000],  loss: 318.651001, mse: 557.210693, mean_q: 7.777756
 1853/2000: episode: 7, duration: 0

...

 1031/10000: episode: 4, duration: 0.946s, episode steps: 150, steps per second: 159, episode reward: -3356.000, mean reward: -22.373 [-50.000, 48.000], mean action: 2.287 [0.000, 3.000],  loss: 605.248158, mse: 907.488090, mean_q: -1.864245
 1110/10000: episode: 5, duration: 0.398s, episode steps:  79, steps per second: 199, episode reward: 468.000, mean reward:  5.924 [-50.000, 64.000], mean action: 1.709 [0.000, 3.000],  loss: 511.679596, mse: 1080.131470, mean_q: -7.386292
 1197/10000: episode: 6, duration: 0.472s, episode steps:  87, steps per second: 184, episode reward: -1084.000, mean reward: -12.460 [-50.000, 32.000], mean action: 0.701 [0.000, 3.000],  loss: 418.814972, mse: 997.068359, mean_q: -6.291780
 1479/10000: episode: 7, duration: 1.294s, episode steps: 282, steps per second: 218, episode reward: -5872.000, mean reward: -20.823 [-50.000, 144.000], mean action: 1.465 [0.000, 3.000],  loss: 482.826904, mse: 1035.965210, mean_q: -6.034113
 1675/10000: episode: 8, duratio

/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   97/10000: episode: 1, duration: 0.283s, episode steps:  97, steps per second: 342, episode reward: 580.000, mean reward:  5.979 [-50.000, 136.000], mean action: 1.423 [0.000, 3.000],  loss: --, mse: --, mean_q: --
  234/10000: episode: 2, duration: 0.092s, episode steps: 137, steps per second: 1482, episode reward: -94.000, mean reward: -0.686 [-50.000, 128.000], mean action: 1.358 [0.000, 3.000],  loss: --, mse: --, mean_q: --
  333/10000: episode: 3, duration: 0.071s, episode steps:  99, steps per second: 1400, episode reward: -1336.000, mean reward: -13.495 [-50.000, 40.000], mean action: 1.616 [0.000, 3.000],  loss: --, mse: --, mean_q: --
  529/10000: episode: 4, duration: 0.139s, episode steps: 196, steps per second: 1412, episode reward: -2870.000, mean reward: -14.643 [-50.000, 160.000], mean action: 1.821 [0.000, 3.000],  loss: --, mse: --, mean_q: --
  670/10000: episode: 5, duration: 0.098s, episode steps: 141, steps per second: 1437, episode reward: -1306.000, mean rewar

/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1071/10000: episode: 7, duration: 1.242s, episode steps: 301, steps per second: 242, episode reward: -7022.000, mean reward: -23.329 [-50.000, 144.000], mean action: 0.874 [0.000, 3.000],  loss: 1058.257135, mse: 11347.533936, mean_q: -73.215160
 1159/10000: episode: 8, duration: 0.411s, episode steps:  88, steps per second: 214, episode reward: 184.000, mean reward:  2.091 [-50.000, 76.000], mean action: 1.989 [0.000, 3.000],  loss: 1060.338745, mse: 11814.556641, mean_q: -77.195229
 1325/10000: episode: 9, duration: 0.765s, episode steps: 166, steps per second: 217, episode reward: -1186.000, mean reward: -7.145 [-50.000, 152.000], mean action: 1.181 [0.000, 3.000],  loss: 999.443237, mse: 11381.389648, mean_q: -74.397835
 1512/10000: episode: 10, duration: 0.840s, episode steps: 187, steps per second: 223, episode reward: -1452.000, mean reward: -7.765 [-50.000, 152.000], mean action: 1.963 [0.000, 3.000],  loss: 939.545349, mse: 10579.209961, mean_q: -70.482445
 1598/10000: episod

# Simulate Game

In [29]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.log2(np.where(grid > 0, grid, 1))
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(100)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    if (i+1) % 10 == 0:
        print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
# display(metrics)
display(metrics.describe())


Game 10 is over.
Game 20 is over.
Game 30 is over.
Game 40 is over.
Game 50 is over.
Game 60 is over.
Game 70 is over.
Game 80 is over.
Game 90 is over.
Game 100 is over.


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.000000,100.000000,100.00000
mean,803.960000,83.200000,0.284232,157.330000,40.790000,44.57000,27.41000,44.560000,1.371752,1.533670,1.377408,1.74171
std,511.688824,55.843938,0.209667,113.790882,41.797321,47.30457,26.16154,44.314377,0.749537,0.887672,1.032266,1.03559
min,140.000000,16.000000,0.000000,45.000000,10.000000,11.00000,2.00000,4.000000,1.000000,1.000000,1.000000,1.00000
25%,512.000000,64.000000,0.086990,88.250000,20.000000,21.00000,14.00000,19.750000,1.000000,1.044091,1.000000,1.03125
50%,650.000000,64.000000,0.279085,122.000000,27.000000,28.00000,20.00000,27.500000,1.045549,1.111046,1.040833,1.17617
75%,1026.000000,128.000000,0.446970,185.750000,42.500000,48.25000,29.00000,51.750000,1.235577,1.683333,1.123575,2.07000
max,2876.000000,256.000000,0.720060,668.000000,264.000000,297.00000,166.00000,273.000000,4.433962,5.033898,6.636364,5.25000


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


